This script is used to divide the dataset into test and enroll parts!
The image is discarded if there are more than one face in the image.

**Note**: this dataset is for testing the algorithm. (*also for training, if the algorithm required*)

In [2]:
# data file root
data_root = 'vgg_face/'

# server setting
host = 'http://127.0.0.1/'
person = 'perons/'
faces = 'faces/'

api_key = 'eb26fa639325f0731166703ebb23723f72396ea2'
appID = '2884084932' # vgg_face app
headers = {'Authorization': 'Token ' + api_key}

In [3]:
import requests, os
from PIL import Image
from pylab import *

detect_url = 'http://172.29.33.44/commands/face_detection/'

def remote_detect(face_filename):
    files = {'image': open(face_filename, 'rb')}
    data = {'appID': appID}
    
    response = requests.post(detect_url, files=files, data=data, headers=headers)
    
    if not str(response.status_code).startswith('2'):
        print('Detect failed!<%s>'%(response.status_code))
        return None
    
    faces = response.json()['faces']
    
    if len(faces) != 1:
        print("Server return error!")
        return None
    
    if len(faces) == 1:
        face = faces[0]
        face_data = [(pixel[0], pixel[1], pixel[2]) for pixel in face['data']]
    
        face_image = Image.new(mode='RGB', size=face['size'])
        face_image.putdata(face_data)
    
        return face_image

In [4]:
import dlib
from PIL import Image
import numpy as np

def local_detect(face_filename):
    detector = dlib.get_frontal_face_detector()
    
    try:
        image = Image.open(face_filename)
        cur_format = image.format
        image = image.convert('RGB')
        imarray = np.array(image)
    
        detections = detector(imarray)
    
        return len(detections) == 1, cur_format
    except:
        return False, None

In [35]:
print(local_detect('data/Aaron_Eckhart/008a9b44d28e03ec64f26e580bb778e2febd7908.jpg'))

(True, 'JPEG')


In [5]:
import os, sys

test_num = 20
test_dest = 'compact_vgg_faces/test/'
enroll_dest = 'compact_vgg_faces/enroll/'

discard = 0
test_total = 0
enroll_total = 0

restart = False

for person in os.listdir(data_root):
    
    test_dir = os.path.join(test_dest, person)
    enroll_dir = os.path.join(enroll_dest, person)
    
    if not restart:
        restart = (person=='6')
        
    if not restart:
        continue
        
    print(person)

    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
        
    if not os.path.exists(enroll_dir):
        os.makedirs(enroll_dir)
    
    count = 0
    for facename in os.listdir(os.path.join(data_root,person)):
        filepath = os.path.join(data_root, person, facename)
        
        valid, cur_format = local_detect(filepath)
        if valid:
            
            try:
                face = remote_detect(filepath)
            except:
                print('Error: ', sys.exc_info()[0])
                continue
        
            if face == None:
                continue
                
            if count < test_num:
                face.save(os.path.join(test_dir, facename), format=cur_format)
                count += 1
                test_total += 1
            else:
                face.save(os.path.join(enroll_dir, facename), format=cur_format)
                enroll_total += 1
                
        else:
            discard += 1
print("Discard: %d; enroll: %d; test: %d"%(discard, enroll_total, test_total))

6
4
8
14
13
33
31
10
17
51


/home/xhoutao/anaconda3/lib/python3.5/site-packages/PIL/Image.py:874: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


53
24
38
9
Error:  <class 'UnicodeEncodeError'>
2
1
45
7
47
22
49
42
Discard: 6888; enroll: 10204; test: 440
